In [5]:
import pandas as pd 

data = pd.read_csv(
    "data/drugs.csv",      # relative python path to subdirectory
    sep=';',           # Tab-separated value file.
    quotechar="\"",        # single quote allowed as quote character
    usecols=['Nº Registro', 'Medicamento', 'Cód. ATC']
)

# Preview the first 5 lines of the loaded data 
data.head()


,Nº Registro,Medicamento,Cód. ATC
0,40537,INACID 25 mg CAPSULAS DURAS,M01AB01
1,7235,PASSIFLORINE SOLUCION ORAL,N05CM
2,18329,TIOBARBITAL BRAUN 1 G,N01AF03
3,33561,DIHYDERGOT COMPRIMIDOS,N02CA01
4,34369,GLUCOSADA GRIFOLS 5% SOLUCION PARA PERFUSION,B05BA03


In [18]:
atc_regs = data.groupby('Cód. ATC')['Nº Registro'].apply(list).reset_index()
print(atc_regs)

     Cód. ATC                                       Nº Registro
0     A01AA01                                    [57745, 57746]
1       A01AB  [38791, 34484, 35528, 12912, 45563, 9957, 29898]
2     A01AB03                                    [84343, 80642]
3     A01AB09                                    [55969, 55962]
4     A01AB19                                           [39783]
...       ...                                               ...
2227    V10XX                              [05322001, 05322002]
2228  V10XX02                                        [03264001]
2229  V10XX03                                       [113873001]
2230  V10XX04                                      [1171226001]
2231   XXXXXX               [75668, 75670, 75659, 75661, 75663]

[2232 rows x 2 columns]


In [19]:
import pysolr

solr = pysolr.Solr('http://librairy.linkeddata.es/data/atc', timeout=2)

def add_num_regs(atc_code,num_regs):
    solr_query = "code_s:"+str(atc_code)
    for result in solr.search(q=solr_query,rows=1):
        result['num_regs_t']=num_regs
        return result
    return {}

updated_drugs = []
num = 0;
for i in atc_regs.itertuples():
    atc_code = i[1]
    num_regs = " ".join(i[2])
    updated_drugs.append(add_num_regs(atc_code,num_regs)) 
    num+=1
    if (len(updated_drugs) > 0 and len(updated_drugs) %100 == 0):
        solr.add(updated_drugs)
        solr.commit()
        updated_drugs = []
        print(num,"drugs updated")

100 drugs updated
200 drugs updated
300 drugs updated
400 drugs updated
500 drugs updated
600 drugs updated
700 drugs updated
800 drugs updated
900 drugs updated
1000 drugs updated
1100 drugs updated
1200 drugs updated
1300 drugs updated
1400 drugs updated
1500 drugs updated
1600 drugs updated
1700 drugs updated
1800 drugs updated
1900 drugs updated
2000 drugs updated
2100 drugs updated
2200 drugs updated
